### Code to get all reservior data in the ouput formate requested by USU with collection system
### this runs for the DataStreams then the following code takes the output csv file and 
### creates the site formated metadata.

In [ ]:
import pandas as pd
import re
from sqlalchemy import create_engine
import urllib

# --- SQL SERVER CONNECTION SETUP ---
server = 'wrt-sql-prod'
database = 'dvrtDB'
username = 'wrtsqlq'
password = '********'

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};DATABASE={database};UID={username};PWD={password}"
)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# --- STEP 1: SQL METADATA QUERY ---
def query_station_metadata():
    sql_query = """
    SELECT  
        LTRIM(RTRIM([COLLECTION_SYSTEM])) AS COLLECTION_SYSTEM,
        LTRIM(RTRIM([collection_sys_description])) AS collection_sys_description,
        [STATION_MASTER].[STATION_ID] AS MasterStationID,
        LTRIM(RTRIM([STATION_MASTER].[STATION_NAME])) AS MasterStationName,
        LTRIM(RTRIM([COLLECTION_STATIONS].[STATION_NAME])) AS CollectionStationName,
        LTRIM(RTRIM([COMMENTS])) AS COMMENTS,
        LTRIM(RTRIM([SiteType])) AS SiteType,
        LTRIM(RTRIM([ANALOG_CHANNEL])) AS ANALOG_CHANNEL,
        LTRIM(RTRIM([SYSTEM_NAME])) AS SYSTEM_NAME,
        LTRIM(RTRIM([DatasetType])) AS DatasetType,
        LTRIM(RTRIM([MEASURING_DEVICE])) AS MEASURING_DEVICE,
        LTRIM(RTRIM([DEVICE_TYPE])) AS DEVICE_TYPE,
        LTRIM(RTRIM([STATUS])) AS STATUS,
        [LAT], [LON],
        LTRIM(RTRIM([DataEntryMethod])) AS DataEntryMethod,
        LTRIM(RTRIM([Telemetry])) AS Telemetry,
        CONCAT('https://waterrights.utah.gov/cgi-bin/dvrtview.exe?Modinfo=StationView&STATION_ID=', [STATION_MASTER].[STATION_ID]) AS StationPage,
        [UNITS_MASTER].[UNITS_ID],
        LTRIM(RTRIM([UNITS_MASTER].[RECORD_TYPE])) AS RECORD_TYPE,
        LTRIM(RTRIM([UNITS_MASTER].[UNITS_DESC_BASE])) AS UNITS_DESC_BASE,
        LTRIM(RTRIM([UNITS_MASTER].[UNITS_DESC_ENTRY])) AS UNITS_DESC_ENTRY,
        [UNITS_MASTER].[UNITS_MULTIPLIER],
        LTRIM(RTRIM([UNITS_MASTER].[UNITS_DESC_REALTIME])) AS UNITS_DESC_REALTIME,
        COUNT([RECORD_YEAR]) AS NoOfYears, 
        MIN([RECORD_YEAR]) AS StartYr, 
        MAX([RECORD_YEAR]) AS EndYr
    FROM [dvrtDB].[dbo].[STATION_MASTER]
    LEFT JOIN [dvrtDB].[dbo].[COLLECTION_SYSTEMS] 
        ON [COLLECTION_SYSTEMS].[collection_sys_id] = [STATION_MASTER].[STATION_ID]
    LEFT JOIN [dvrtDB].[dbo].[COLLECTION_STATIONS] 
        ON [STATION_MASTER].[CAPTURE_SEQ_NO] = [COLLECTION_STATIONS].[SEQ_NO]
    JOIN [dvrtDB].[dbo].[UNITS_MASTER] 
        ON [STATION_MASTER].[UNITS_ID] = [UNITS_MASTER].[UNITS_ID]
    LEFT JOIN [dvrtDB].[dbo].[DAILY_RECORDS] 
        ON [STATION_MASTER].[STATION_ID] = [DAILY_RECORDS].[STATION_ID]
    WHERE 
        [STATUS] = 'A' AND
        [DatasetType] = 'Observational' AND
        [DataEntryMethod] != 'Manual' AND
        [DataEntryMethod] IS NOT NULL AND
        ([LAT] IS NOT NULL OR [LON] IS NOT NULL) AND
        ([LON] > '-115' OR [LON] < '36') AND
        [LAT] > 0 AND
        (
            [STATION_MASTER].[STATION_NAME] LIKE '%Reservoir%' OR 
            [COLLECTION_STATIONS].[STATION_NAME] LIKE '%Reservoir%'
        )
    GROUP BY
        [COLLECTION_SYSTEM], [collection_sys_description], [STATION_MASTER].[STATION_ID],
        [STATION_MASTER].[STATION_NAME], [COLLECTION_STATIONS].[STATION_NAME],
        [COMMENTS], [SiteType], [ANALOG_CHANNEL], [SYSTEM_NAME], [DatasetType],
        [MEASURING_DEVICE], [DEVICE_TYPE], [STATUS], [LAT], [LON],
        [DataEntryMethod], [Telemetry],
        [UNITS_MASTER].[UNITS_ID], [UNITS_MASTER].[RECORD_TYPE],
        [UNITS_MASTER].[UNITS_DESC_BASE], [UNITS_MASTER].[UNITS_DESC_ENTRY],
        [UNITS_MASTER].[UNITS_MULTIPLIER], [UNITS_MASTER].[UNITS_DESC_REALTIME]
    ORDER BY [STATION_MASTER].[STATION_ID]
    """
    return pd.read_sql(sql_query, engine)

# --- STEP 2: FINAL EXPORT LOGIC WITH SPLIT SITE ID ---
def generate_final_export(df):
    df = df[df["SiteType"].isin(["Reservoir", "Reservoir Release"])].copy()

    # Tag which rows are release-type
    df["IsRelease"] = df["MasterStationName"].str.upper().str.contains("RELEASE")

    # Extract base name (remove USBR etc.)
    def extract_base_name(name):
        name = str(name).upper()
        name = re.sub(r'\b(USBR|USGS|RESERVOIR|RELEASE|CONTENTS|ELEVATION|STORAGE|POOL|EVAPORATION|LEVEL|GAGE HEIGHT)\b', '', name)
        name = re.sub(r'[^A-Z0-9 ]+', '', name)
        name = re.sub(r'\s+', ' ', name).strip()
        return name.split()[0].title() if name else "Unknown"

    df["ReservoirRootName"] = df["MasterStationName"].apply(extract_base_name)

    # === Split ===
    release_df = df[df["IsRelease"]].copy()
    nonrelease_df = df[~df["IsRelease"]].copy()

    # Assign PD### to non-release groups by root name
    unique_nonreleases = nonrelease_df["ReservoirRootName"].unique()
    nonrelease_group_map = {name: f"PD{str(i+1).zfill(3)}" for i, name in enumerate(unique_nonreleases)}
    nonrelease_df["SiteID (New)"] = nonrelease_df["ReservoirRootName"].map(nonrelease_group_map)

    # Assign PD### to each release row independently
    release_df = release_df.reset_index(drop=True)
    release_df["SiteID (New)"] = ["PD" + str(i + len(nonrelease_group_map) + 1).zfill(3) for i in range(len(release_df))]

    # Merge back
    final_df = pd.concat([nonrelease_df, release_df], ignore_index=True)

    # Build NewSiteName
    def make_site_name(row):
        base = row["ReservoirRootName"]
        system = row["SYSTEM_NAME"].strip().title()
        if row["IsRelease"]:
            return f"{base} River Below {base} Reservoir, {system}, Near {base}"
        else:
            return f"{base} Reservoir, {system}, Near {base}"

    final_df["NewSiteName"] = final_df.apply(make_site_name, axis=1)

    # Sort and assign DS##
    final_df = final_df.sort_values(
        by=["ReservoirRootName", "IsRelease", "MasterStationName"],
        ascending=[True, True, True]
    ).reset_index(drop=True)
    final_df["DataStreamID"] = ["DS" + str(i+1).zfill(2) for i in range(len(final_df))]

    # Final export
    export = final_df[[
        "NewSiteName",
        "SiteID (New)",
        "DataStreamID",
        "MasterStationName",
        "MasterStationID",
        "UNITS_DESC_ENTRY",
        "CollectionStationName",
        "COLLECTION_SYSTEM"
    ]].rename(columns={
        "MasterStationName": "DIVERT_STATION_NAME (old)",
        "MasterStationID": "Station_ID (old)",
        "COLLECTION_SYSTEM": "CollectionSystemName"
    })

    export.to_csv("Reservior_Station_Datastream_Metadata_HydroServer_USU_20250507.csv", index=False)
    print("✅ Saved: Reservior_Station_Datastream_Metadata_HydroServer_USU_20250507.csv")

# --- MAIN ---
def main():
    print("🔎 Querying metadata...")
    metadata_df = query_station_metadata()
    print(f"📦 {len(metadata_df)} rows retrieved.")

    print("🧠 Structuring export with release/non-release logic...")
    generate_final_export(metadata_df)

    print("✅ Done.")

if __name__ == "__main__":
    main()

### Following code takes the output datastreams csv file from above ""Reservior_Station_Datastream_Metadata_HydroServer_USU_20250507.csv" and creates the site formated metadata directed by the USU team

In [ ]:


import pandas as pd

# Load the structured metadata
df = pd.read_csv("Reservior_Station_Datastream_Metadata_HydroServer_USU_20250507.csv")

# Load the original SQL metadata (used to retrieve LAT, LON, SYSTEM_NAME)
original_metadata = query_station_metadata()  # or read from a saved CSV if you prefer

# Match rows by Station_ID
merged = df.merge(
    original_metadata[["MasterStationID", "LAT", "LON", "SYSTEM_NAME","COLLECTION_SYSTEM"]],
    left_on="Station_ID (old)",
    right_on="MasterStationID",
    how="left"
)

# Determine SiteType
merged["SiteType"] = merged["DIVERT_STATION_NAME (old)"].str.upper().apply(
    lambda x: "Reservoir Release" if "RELEASE" in x else "Reservoir"
)

# Group by SiteID and take the first entry (since NewSiteName, SiteID, LAT/LON are consistent per site)
site_meta = (
    merged.groupby("SiteID (New)")
    .first()
    .reset_index()
    [[
        "NewSiteName",
        "SiteID (New)",
        "SiteType",
        "LAT",
        "LON",
        "SYSTEM_NAME",
        "COLLECTION_SYSTEM"
    ]]
)

# OPTIONAL: You can add County and River if you have a function or mapping
# site_meta["County"] = ...
# site_meta["River"] = ...

# Rename SYSTEM_NAME and COLLECTION_SYSTEM for export
site_meta = site_meta.rename(columns={
    "SYSTEM_NAME": "Workspace",
    "COLLECTION_SYSTEM": "Collection_Systems"
})

# Save output
site_meta.to_csv("Reservior_Station_Sites_Metadata_HydroServer_USU_20250507.csv", index=False)
print("✅ Saved: Reservior_Station_Sites_Metadata_HydroServer_USU_20250507.csv")